In [2]:
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Masking
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences




In [3]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')  # May improve speed on some CPUs


In [4]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')  # May improve speed on some CPUs


In [5]:
def read_trc(fpath):
    # read metadata in file header
    df_meta = pd.read_csv(fpath, delimiter='\t', header=0, skiprows=1, nrows=1)
    meta = df_meta.iloc[0].to_dict()
    fps = meta['DataRate']

    # read marker location names
    markers_df = pd.read_csv(fpath, delimiter='\t', header=None, skiprows=3, nrows=1)
    markers = markers_df.iloc[0].dropna().to_numpy()[2:]

    # read marker XYZ locations
    df = pd.read_csv(fpath, delimiter='\t', header=0, skiprows=4)
    df.rename(columns=dict(zip(df.columns[:2], ('n', 't'))), inplace=True)
    df.dropna(how='all', axis=1, inplace=True)

    N = df.shape[0]
    M = len(markers)
    xyz = df.iloc[:,2:].to_numpy().reshape((N, M, 3))
    xyz[:,:,[0,1,2]] = xyz[:,:,[2,1,0]]

    return fps, markers, xyz


def read_mot(fpath):
    with open(fpath, 'r') as f:
        # count = 0
        line = f.readline().strip()
        # while line and line.strip() != 'endheader':
        while line.lower() != 'endheader':
            line = f.readline().strip()
            # count += 1

        # df = pd.read_csv(f, delimiter='\t', header=0, skiprows=count-3)
        df = pd.read_csv(f, delimiter='\t', header=0)

    return df

In [6]:
# # Define function to normalize joint angles across all data
# def normalize_data(data_list):
#     """
#     Normalizes a list of numpy arrays using MinMax scaling across all features.
#     Arguments:
#         data_list: List of numpy arrays where each array corresponds to one time series.
#     Returns:
#         normalized_data: List of normalized numpy arrays.
#     """
#     # Flatten all data for joint normalization across the dataset
#     all_data = np.array(data_list)
    
#     # Apply MinMaxScaler
#     scaler = MinMaxScaler()
#     scaler.fit(all_data)  # Fit scaler on all joint angles across the dataset
    
#     # Normalize each time series
#     normalized_data = [scaler.transform(data) for data in data_list]
#     return normalized_data, scaler

# # Define function to pad time series to the same length
# def pad_time_series(data_list, max_length=None):
#     """
#     Pads all time series in data_list to the same length.
#     Arguments:
#         data_list: List of numpy arrays where each array corresponds to one time series.
#         max_length: If None, pad to the longest time series. Otherwise, pad to max_length.
#     Returns:
#         padded_data: Numpy array of shape (num_samples, max_length, num_features).
#     """
#     if max_length is None:
#         max_length = max(len(data) for data in data_list)  # Find the longest time series
#     padded_data = pad_sequences(data_list, maxlen=max_length, padding="post", dtype="float32")
#     return padded_data

In [7]:
def z_score_normalize(data_list):
    """
    Applies z-score normalization to a list of time series arrays.
    Arguments:
        data_list: List of numpy arrays, each of shape (num_timesteps, num_features).
    Returns:
        normalized_data: List of z-score normalized arrays.
        mean: Mean of each feature across all time series.
        std: Standard deviation of each feature across all time series.
    """
    # Concatenate all data to compute global mean and std
    all_data = np.concatenate(data_list, axis=0)
    mean = np.mean(all_data, axis=0)
    std = np.std(all_data, axis=0)
    
    # Normalize each array
    normalized_data = [(data - mean) / (std + 1e-8) for data in data_list]
    return normalized_data, mean, std

def prepare_lstm_input(subject_dict):
    """
    Combines multiple time series for each subject along the time axis, applies z-score normalization,
    and pads them for LSTM input.
    Arguments:
        subject_dict: Dictionary where keys are subject IDs and values are lists of 6 numpy arrays.
                      Each numpy array corresponds to a time series with shape (num_timesteps, num_features).
    Returns:
        padded_input: A numpy array of shape (num_subjects, max_timesteps, num_features).
        subject_ids: List of subject IDs in the order they appear in the padded_input array.
        mean: Mean of features used for z-score normalization.
        std: Standard deviation of features used for z-score normalization.
    """
    # Sort the dictionary by keys to ensure consistent ordering
    subject_ids = sorted(subject_dict.keys())
    
    # Extract and concatenate the 6 time series for each subject along the time axis
    concatenated_series = [
        np.concatenate(subject_dict[subject_id], axis=0) for subject_id in subject_ids
    ]
    
    # Apply z-score normalization to the concatenated series
    normalized_series, mean, std = z_score_normalize(concatenated_series)
    
    # Determine the maximum length for padding
    max_length = max(series.shape[0] for series in normalized_series)
    
    # Pad all normalized time series to the same length
    padded_input = pad_sequences(normalized_series, maxlen=max_length, padding="post", dtype="float32")
    
    return padded_input, subject_ids, mean, std

In [8]:
def get_label_for_participant_trial(class_df, pid, trial):
    """
    Fetches the label for a given participant and trial based on class_info.csv.
    
    Parameters:
        participant_id (str): The ID of the participant.
        trial (str): The trial name or ID.
    
    Returns:
        int: Classification label (0 or 1).
    """
    label = class_df.loc[class_df['ID'] == pid, 'Class'].values[0]
    return label

In [14]:
# Directory containing .mot files
root_dir = '/Users/scovitz/datadir/opencap_data'  # path to root data dir 
labels_csv = '/Users/scovitz/datadir/class_info.csv'
class_df = pd.read_csv(labels_csv)

time_series_data = {}  # dictionary of pid, timeseries pairs 

expected_columns = [
        'pelvis_tilt', 'pelvis_list', 'pelvis_rotation', 'pelvis_tx',
        'pelvis_ty', 'pelvis_tz', 'hip_flexion_r', 'hip_adduction_r',
        'hip_rotation_r', 'knee_angle_r', 'ankle_angle_r',
        'subtalar_angle_r', 'mtp_angle_r', 'hip_flexion_l', 'hip_adduction_l',
        'hip_rotation_l', 'knee_angle_l', 'ankle_angle_l',
        'subtalar_angle_l', 'mtp_angle_l', 'lumbar_extension', 'lumbar_bending',
        'lumbar_rotation', 'arm_flex_r', 'arm_add_r', 'arm_rot_r',
        'elbow_flex_r', 'pro_sup_r', 'arm_flex_l', 'arm_add_l', 'arm_rot_l',
        'elbow_flex_l', 'pro_sup_l'
    ]

# Loop through all .mot files
for i in range(129):
        # ensure going in order!
        num = str(i+1).zfill(3)

        for path in Path(root_dir).rglob(f"P{num}*/**/*.mot"): 
            
            # remove upper extremity trials 
            if "brooke" in str(path) or "curls" in str(path) or "arm_rom" in str(path):
                continue 
            
            trial = str(path).split('/')[-3]
            pid = str(path).split('/')[-4]
            activity = str(path).split('/')[-2]
            #print(str(path))
            # Load the .mot file
            
            data = read_mot(str(path))
            data = data[200:]
            #print(data.shape)

            # Ensure the file has the expected columns
            if not all(col in data.columns for col in expected_columns + ['time']):
                raise ValueError(f"File {path} does not contain the expected columns.")
            
            # Drop the 'time' column and keep only relevant features
            data = np.array(data[expected_columns])
            
            if pid not in time_series_data:
                time_series_data[pid + '_' + trial] = []

            time_series_data[(pid + '_' + trial)].append(data) # value = list of timeseries arrays of num_timesteps x num_feats
            print(data.shape, pid, trial, str(path))




(635, 33) P001 0 /Users/scovitz/datadir/opencap_data/P001/0/walk/walk.mot
(332, 33) P001 0 /Users/scovitz/datadir/opencap_data/P001/0/tug/tug.mot
(1838, 33) P001 0 /Users/scovitz/datadir/opencap_data/P001/0/toe_stand/toe_stand.mot
(652, 33) P001 0 /Users/scovitz/datadir/opencap_data/P001/0/5xsts/5xsts.mot
(140, 33) P001 0 /Users/scovitz/datadir/opencap_data/P001/0/jump/jump.mot
(273, 33) P001 0 /Users/scovitz/datadir/opencap_data/P001/0/run/run.mot
(642, 33) P002 0 /Users/scovitz/datadir/opencap_data/P002/0/walk/walk.mot
(1119, 33) P002 0 /Users/scovitz/datadir/opencap_data/P002/0/tug/tug.mot
(1929, 33) P002 0 /Users/scovitz/datadir/opencap_data/P002/0/toe_stand/toe_stand.mot
(875, 33) P002 0 /Users/scovitz/datadir/opencap_data/P002/0/5xsts/5xsts.mot
(161, 33) P002 0 /Users/scovitz/datadir/opencap_data/P002/0/jump/jump.mot
(456, 33) P002 0 /Users/scovitz/datadir/opencap_data/P002/0/run/run.mot
(625, 33) P003 0 /Users/scovitz/datadir/opencap_data/P003/0/walk/walk.mot
(948, 33) P003 0 /U

(908, 33) P018 0 /Users/scovitz/datadir/opencap_data/P018/0/walk/walk.mot
(1318, 33) P018 0 /Users/scovitz/datadir/opencap_data/P018/0/tug/tug.mot
(1896, 33) P018 0 /Users/scovitz/datadir/opencap_data/P018/0/toe_stand/toe_stand.mot
(920, 33) P018 0 /Users/scovitz/datadir/opencap_data/P018/0/5xsts/5xsts.mot
(260, 33) P018 0 /Users/scovitz/datadir/opencap_data/P018/0/jump/jump.mot
(147, 33) P018 0 /Users/scovitz/datadir/opencap_data/P018/0/run/run.mot
(866, 33) P019 7 /Users/scovitz/datadir/opencap_data/P019/7/walk/walk.mot
(1452, 33) P019 7 /Users/scovitz/datadir/opencap_data/P019/7/tug/tug.mot
(1781, 33) P019 7 /Users/scovitz/datadir/opencap_data/P019/7/toe_stand/toe_stand.mot
(611, 33) P019 7 /Users/scovitz/datadir/opencap_data/P019/7/5xsts/5xsts.mot
(160, 33) P019 7 /Users/scovitz/datadir/opencap_data/P019/7/jump/jump.mot
(461, 33) P019 7 /Users/scovitz/datadir/opencap_data/P019/7/run/run.mot
(1002, 33) P019 118 /Users/scovitz/datadir/opencap_data/P019/118/walk/walk.mot
(1350, 33) P0

(456, 33) P033 7 /Users/scovitz/datadir/opencap_data/P033/7/5xsts/5xsts.mot
(117, 33) P033 7 /Users/scovitz/datadir/opencap_data/P033/7/jump/jump.mot
(372, 33) P033 7 /Users/scovitz/datadir/opencap_data/P033/7/run/run.mot
(961, 33) P034 7 /Users/scovitz/datadir/opencap_data/P034/7/walk/walk.mot
(1720, 33) P034 7 /Users/scovitz/datadir/opencap_data/P034/7/tug/tug.mot
(1132, 33) P034 7 /Users/scovitz/datadir/opencap_data/P034/7/5xsts/5xsts.mot
(164, 33) P034 7 /Users/scovitz/datadir/opencap_data/P034/7/jump/jump.mot
(778, 33) P034 7 /Users/scovitz/datadir/opencap_data/P034/7/run/run.mot
(1069, 33) P035 406 /Users/scovitz/datadir/opencap_data/P035/406/walk/walk.mot
(1249, 33) P035 406 /Users/scovitz/datadir/opencap_data/P035/406/tug/tug.mot
(2060, 33) P035 406 /Users/scovitz/datadir/opencap_data/P035/406/toe_stand/toe_stand.mot
(1638, 33) P035 406 /Users/scovitz/datadir/opencap_data/P035/406/5xsts/5xsts.mot
(381, 33) P035 406 /Users/scovitz/datadir/opencap_data/P035/406/jump/jump.mot
(639

(706, 33) P047 8 /Users/scovitz/datadir/opencap_data/P047/8/walk/walk.mot
(1029, 33) P047 8 /Users/scovitz/datadir/opencap_data/P047/8/tug/tug.mot
(1868, 33) P047 8 /Users/scovitz/datadir/opencap_data/P047/8/toe_stand/toe_stand.mot
(558, 33) P047 8 /Users/scovitz/datadir/opencap_data/P047/8/5xsts/5xsts.mot
(161, 33) P047 8 /Users/scovitz/datadir/opencap_data/P047/8/jump/jump.mot
(311, 33) P047 8 /Users/scovitz/datadir/opencap_data/P047/8/run/run.mot
(1518, 33) P048 8 /Users/scovitz/datadir/opencap_data/P048/8/walk/walk.mot
(1480, 33) P048 8 /Users/scovitz/datadir/opencap_data/P048/8/tug/tug.mot
(1960, 33) P048 8 /Users/scovitz/datadir/opencap_data/P048/8/5xsts/5xsts.mot
(720, 33) P048 8 /Users/scovitz/datadir/opencap_data/P048/8/run/run.mot
(135, 33) P049 8 /Users/scovitz/datadir/opencap_data/P049/8/walk/walk.mot
(255, 33) P049 8 /Users/scovitz/datadir/opencap_data/P049/8/tug/tug.mot
(1867, 33) P049 8 /Users/scovitz/datadir/opencap_data/P049/8/toe_stand/toe_stand.mot
(512, 33) P049 8 /

(649, 33) P063 8 /Users/scovitz/datadir/opencap_data/P063/8/5xsts/5xsts.mot
(400, 33) P063 8 /Users/scovitz/datadir/opencap_data/P063/8/jump/jump.mot
(187, 33) P063 8 /Users/scovitz/datadir/opencap_data/P063/8/run/run.mot
(675, 33) P064 406 /Users/scovitz/datadir/opencap_data/P064/406/walk/walk.mot
(976, 33) P064 406 /Users/scovitz/datadir/opencap_data/P064/406/tug/tug.mot
(2006, 33) P064 406 /Users/scovitz/datadir/opencap_data/P064/406/toe_stand/toe_stand.mot
(1462, 33) P064 406 /Users/scovitz/datadir/opencap_data/P064/406/5xsts/5xsts.mot
(206, 33) P064 406 /Users/scovitz/datadir/opencap_data/P064/406/jump/jump.mot
(618, 33) P064 406 /Users/scovitz/datadir/opencap_data/P064/406/run/run.mot
(527, 33) P064 8 /Users/scovitz/datadir/opencap_data/P064/8/walk/walk.mot
(927, 33) P064 8 /Users/scovitz/datadir/opencap_data/P064/8/tug/tug.mot
(1815, 33) P064 8 /Users/scovitz/datadir/opencap_data/P064/8/toe_stand/toe_stand.mot
(531, 33) P064 8 /Users/scovitz/datadir/opencap_data/P064/8/5xsts/5xs

(534, 33) P078 8 /Users/scovitz/datadir/opencap_data/P078/8/walk/walk.mot
(1001, 33) P078 8 /Users/scovitz/datadir/opencap_data/P078/8/tug/tug.mot
(1840, 33) P078 8 /Users/scovitz/datadir/opencap_data/P078/8/toe_stand/toe_stand.mot
(727, 33) P078 8 /Users/scovitz/datadir/opencap_data/P078/8/5xsts/5xsts.mot
(184, 33) P078 8 /Users/scovitz/datadir/opencap_data/P078/8/jump/jump.mot
(831, 33) P078 8 /Users/scovitz/datadir/opencap_data/P078/8/run/run.mot
(780, 33) P079 8 /Users/scovitz/datadir/opencap_data/P079/8/walk/walk.mot
(1478, 33) P079 8 /Users/scovitz/datadir/opencap_data/P079/8/tug/tug.mot
(1960, 33) P079 8 /Users/scovitz/datadir/opencap_data/P079/8/toe_stand/toe_stand.mot
(1349, 33) P079 8 /Users/scovitz/datadir/opencap_data/P079/8/5xsts/5xsts.mot
(218, 33) P079 8 /Users/scovitz/datadir/opencap_data/P079/8/jump/jump.mot
(762, 33) P079 8 /Users/scovitz/datadir/opencap_data/P079/8/run/run.mot
(644, 33) P080 11 /Users/scovitz/datadir/opencap_data/P080/11/walk/walk.mot
(1066, 33) P080

(654, 33) P093 118 /Users/scovitz/datadir/opencap_data/P093/118/walk/walk.mot
(1105, 33) P093 118 /Users/scovitz/datadir/opencap_data/P093/118/tug/tug.mot
(1599, 33) P093 118 /Users/scovitz/datadir/opencap_data/P093/118/5xsts/5xsts.mot
(32, 33) P093 118 /Users/scovitz/datadir/opencap_data/P093/118/jump/jump.mot
(413, 33) P093 118 /Users/scovitz/datadir/opencap_data/P093/118/run/run.mot
(677, 33) P093 119 /Users/scovitz/datadir/opencap_data/P093/119/walk/walk.mot
(1230, 33) P093 119 /Users/scovitz/datadir/opencap_data/P093/119/tug/tug.mot
(373, 33) P093 119 /Users/scovitz/datadir/opencap_data/P093/119/toe_stand/toe_stand.mot
(1486, 33) P093 119 /Users/scovitz/datadir/opencap_data/P093/119/5xsts/5xsts.mot
(260, 33) P093 119 /Users/scovitz/datadir/opencap_data/P093/119/jump/jump.mot
(379, 33) P093 119 /Users/scovitz/datadir/opencap_data/P093/119/run/run.mot
(895, 33) P094 118 /Users/scovitz/datadir/opencap_data/P094/118/walk/walk.mot
(1482, 33) P094 118 /Users/scovitz/datadir/opencap_data

(655, 33) P105 119 /Users/scovitz/datadir/opencap_data/P105/119/walk/walk.mot
(697, 33) P105 119 /Users/scovitz/datadir/opencap_data/P105/119/tug/tug.mot
(1932, 33) P105 119 /Users/scovitz/datadir/opencap_data/P105/119/toe_stand/toe_stand.mot
(1567, 33) P105 119 /Users/scovitz/datadir/opencap_data/P105/119/5xsts/5xsts.mot
(69, 33) P105 119 /Users/scovitz/datadir/opencap_data/P105/119/jump/jump.mot
(125, 33) P105 119 /Users/scovitz/datadir/opencap_data/P105/119/run/run.mot
(652, 33) P106 119 /Users/scovitz/datadir/opencap_data/P106/119/walk/walk.mot
(780, 33) P106 119 /Users/scovitz/datadir/opencap_data/P106/119/tug/tug.mot
(2090, 33) P106 119 /Users/scovitz/datadir/opencap_data/P106/119/toe_stand/toe_stand.mot
(841, 33) P106 119 /Users/scovitz/datadir/opencap_data/P106/119/5xsts/5xsts.mot
(0, 33) P106 119 /Users/scovitz/datadir/opencap_data/P106/119/jump/jump.mot
(291, 33) P106 119 /Users/scovitz/datadir/opencap_data/P106/119/run/run.mot
(636, 33) P107 119 /Users/scovitz/datadir/openca

(853, 33) P125 455 /Users/scovitz/datadir/opencap_data/P125/455/walk/walk.mot
(994, 33) P125 455 /Users/scovitz/datadir/opencap_data/P125/455/tug/tug.mot
(1951, 33) P125 455 /Users/scovitz/datadir/opencap_data/P125/455/toe_stand/toe_stand.mot
(969, 33) P125 455 /Users/scovitz/datadir/opencap_data/P125/455/5xsts/5xsts.mot
(206, 33) P125 455 /Users/scovitz/datadir/opencap_data/P125/455/jump/jump.mot
(365, 33) P125 455 /Users/scovitz/datadir/opencap_data/P125/455/run/run.mot
(802, 33) P126 455 /Users/scovitz/datadir/opencap_data/P126/455/walk/walk.mot
(776, 33) P126 455 /Users/scovitz/datadir/opencap_data/P126/455/tug/tug.mot
(2000, 33) P126 455 /Users/scovitz/datadir/opencap_data/P126/455/toe_stand/toe_stand.mot
(960, 33) P126 455 /Users/scovitz/datadir/opencap_data/P126/455/5xsts/5xsts.mot
(265, 33) P126 455 /Users/scovitz/datadir/opencap_data/P126/455/jump/jump.mot
(287, 33) P126 455 /Users/scovitz/datadir/opencap_data/P126/455/run/run.mot
(792, 33) P127 455 /Users/scovitz/datadir/open

In [ ]:
len(time_series_data)
time_series_data

In [ ]:
# Prepare the LSTM input
lstm_input, subject_ids, mean, std = prepare_lstm_input(time_series_data)

# Check the results
print("LSTM Input Shape:", lstm_input.shape)  # (162, max_timesteps, 35)
print("Subject IDs:", subject_ids[:5])  # subject ID ordering, should be in order! 
print("Mean Shape:", mean.shape)  # shape of mean: (35,)
print("Std Shape:", std.shape)  # shape of std: (35,)




In [ ]:
# Convert labels to a numpy array
labels = []
for sid in subject_ids:
    pid = sid.split('_')[0]
    trial = sid.split('_')[1]
    label = get_label_for_participant_trial(class_df, pid, trial)
    labels.append(label)

In [ ]:
# convert to numpy array
labels = np.array(labels)

In [ ]:
# Define LSTM model
# def build_lstm_model_1(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking for padded values
#         LSTM(64, return_sequences=True),  # First LSTM layer
#         Dropout(0.3),  # Dropout for regularization
#         LSTM(32),  # Second LSTM layer
#         Dropout(0.3),
#         Dense(16, activation='relu'),  # Fully connected layer
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model


In [ ]:
# def build_lstm_model_2(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking for padded values
#         LSTM(128, return_sequences=True),  # First LSTM layer with increased units
#         BatchNormalization(),  # Batch normalization to stabilize training
#         Dropout(0.2),  # Reduced dropout rate
#         LSTM(64, return_sequences=False),  # Second LSTM layer
#         BatchNormalization(),  # Batch normalization after second LSTM layer
#         Dropout(0.2),
#         Dense(32, activation='relu'),  # Increased fully connected layer size
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.0001),  # Reduced learning rate
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model

In [ ]:
# data loader
def data_generator(X, y, batch_size):
    num_samples = len(X)
    while True:
        for offset in range(0, num_samples, batch_size):
            X_batch = X[offset:offset+batch_size]
            y_batch = y[offset:offset+batch_size]
            yield np.array(X_batch), np.array(y_batch)


In [ ]:


# # Ensure consistent shuffling for reproducibility
# np.random.seed(42)

# def manual_train_val_test_split(data, labels, train_ratio=0.8, val_ratio=0.1):
#     """
#     Manually splits data and labels into training, validation, and testing sets with the given ratios.
#     Arguments:
#         data: Preprocessed time series data (numpy array).
#         labels: Corresponding labels (numpy array).
#         train_ratio: Proportion of data to use for training (default 0.7).
#         val_ratio: Proportion of data to use for validation (default 0.2).
#     Returns:
#         X_train, X_val, X_test, y_train, y_val, y_test: Split data and labels.
#     """
#     # Ensure train_ratio + val_ratio < 1
#     assert train_ratio + val_ratio < 1, "Train and validation ratios must sum to less than 1."
    
#     # Shuffle the indices
#     indices = np.arange(len(data))
#     np.random.shuffle(indices)
    
#     # Calculate the split points
#     train_split = int(len(data) * train_ratio)
#     val_split = int(len(data) * (train_ratio + val_ratio))
    
#     # Split the data and labels
#     train_indices = indices[:train_split]
#     val_indices = indices[train_split:val_split]
#     test_indices = indices[val_split:]
    
#     X_train = data[train_indices]
#     X_val = data[val_indices]
#     X_test = data[test_indices]
#     y_train = labels[train_indices]
#     y_val = labels[val_indices]
#     y_test = labels[test_indices]
    
#     return X_train, X_val, X_test, y_train, y_val, y_test


In [ ]:
# ### Perform the manual train-test split
# # Call the function
# X_train, X_val, X_test, y_train, y_val, y_test = manual_train_val_test_split(
#     np.array(lstm_input), np.array(labels), train_ratio=0.8, val_ratio=0.1)
# # Check the shapes of the resulting datasets
# print("Training data shape:", X_train.shape)
# print("Training labels shape:", y_train.shape)
# print("Testing data shape:", X_test.shape)
# print("Testing labels shape:", y_test.shape)


# # Convert to numpy arrays
# X_train = np.array(X_train)
# X_val = np.array(X_val)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_val = np.array(y_val)
# y_test = np.array(y_test)

# # Define input shape (timepoints, features)
# input_shape = (X_train.shape[1], X_train.shape[2])

# # # Build model
# # model = build_lstm_model(input_shape)

# # # Train the model
# # history = model.fit(
# #     X_train, y_train,
# #     validation_data=(X_val, y_val),
# #     epochs=50,
# #     batch_size=16,
# #     verbose=1
# # )




In [ ]:
# TRY 3 
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Masking, LSTM, Dropout, Dense, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np

# # Define LSTM model with improvements
# def build_lstm_model(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking for padded values
#         LSTM(128, return_sequences=True),  # Increased units in the first LSTM layer
#         BatchNormalization(),  # Batch Normalization
#         Dropout(0.4),  # Increased Dropout for regularization
#         LSTM(64),  # Second LSTM layer with reduced units
#         BatchNormalization(),  # Batch Normalization
#         Dropout(0.4),
#         Dense(32, activation='relu'),  # Increased units in fully connected layer
#         Dropout(0.4),  # Additional dropout
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.0001),  # Reduced learning rate
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model

# # Build the model
# input_shape = (X_train.shape[1], X_train.shape[2])  # Define input shape based on your data
# model = build_lstm_model(input_shape)

# # Compute class weights for handling class imbalance
# class_weights = compute_class_weight(
#     'balanced',
#     classes=np.unique(y_train),
#     y=y_train
# )
# class_weights = dict(enumerate(class_weights))

# # Add EarlyStopping for regularization
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Train the model with validation and early stopping
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=50,
#     batch_size=16,
#     class_weight=class_weights,
#     callbacks=[early_stopping],
#     verbose=1
# )

# # Evaluate the model on test data
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
# print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# # THE LATEST LSTM

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout, BatchNormalization, Input
from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# # Define the LSTM model
# def build_lstm_model(input_shape):
#     model = Sequential([
#         Input(shape=input_shape),
#         LSTM(256, return_sequences=True),  # Increase units
#         Dense(128, activation='relu'),  # Add a dense layer
#         LSTM(128, return_sequences=True),  # Second LSTM layer
#         Dense(64, activation='relu'),  # Add another dense layer
#         LSTM(64),  # Final LSTM layer
#         Dense(32, activation='relu'),  # Fully connected layer
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Higher LR
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model




# Split data into train, validation, and test sets - 80, 10, 10
def split_data(X, y, test_size=0.1, val_size=0.1):
    """
    Splits the data into training, validation, and testing sets.
    Arguments:
        X: Input features (numpy array).
        y: Labels (numpy array).
        test_size: Proportion of data for the test set.
        val_size: Proportion of data for the validation set from the training set.
    Returns:
        X_train, X_val, X_test, y_train, y_val, y_test3
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


# Split data
X_train, X_val, X_test, y_train, y_val, y_test = split_data(lstm_input, labels)

# # Build the model
# input_shape = (X_train.shape[1], X_train.shape[2])  # (timesteps, features)
# model = build_lstm_model(input_shape)

# # Increase batch size and train
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=500,
#     batch_size=32,  # Increased batch size
#     verbose=1
# )


# final_train_accuracy = history.history['accuracy'][-1]
# final_val_accuracy = history.history['val_accuracy'][-1]

# print(f"Final Training Accuracy: {final_train_accuracy}")
# print(f"Final Validation Accuracy: {final_val_accuracy}")

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
# print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# # Classification report
# y_pred = (model.predict(X_test) > 0.5).astype(int)
# # print(classification_report(y_test, y_pred))


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Input
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import LearningRateScheduler

# def build_lstm_model(input_shape):
#     model = Sequential([
#         Input(shape=input_shape),
#         LSTM(256, return_sequences=True),  # Larger capacity
#         LSTM(128, return_sequences=True),
#         LSTM(128, return_sequences=True),
#         LSTM(64),
#         Dense(32, activation='relu'),
#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.0001),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model


# # Build model
# input_shape = (X_train.shape[1], X_train.shape[2])  # Define input shape from the training data
# model = build_lstm_model(input_shape)


# # Train the model with class weights
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_test, y_test),
#     epochs=500,
#     batch_size=16,
#     verbose=1
# )




In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout, BatchNormalization
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# # Build an LSTM model optimized for overfitting to the training set
# def build_overfitting_lstm(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking padded values
#         LSTM(128, return_sequences=True, activation='relu'),  # First LSTM layer
#         LSTM(64, return_sequences=True, activation='relu'),   # Second LSTM layer
#         LSTM(32, activation='relu'),                         # Third LSTM layer
#         Dense(1, activation='sigmoid')                       # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.001),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model

# # Train the model on training data only
# input_shape = (X_train.shape[1], X_train.shape[2])  # Define input shape from the training data
# model = build_overfitting_lstm(input_shape)

# # Train the model for an extended number of epochs
# history = model.fit(
#     X_train, y_train,
#     epochs=100,  # Increase epochs for training
#     batch_size=16,
#     verbose=1
# )

# # Plot training loss over epochs
# plt.plot(history.history['loss'], label='Train Loss')
# plt.title('Training Loss Over Epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()


In [ ]:
# import matplotlib.pyplot as plt

# # Extract the loss values from the history object
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']

# # Print loss values for each epoch
# print("Training Loss per Epoch:")
# for epoch, loss in enumerate(train_loss):
#     print(f"Epoch {epoch+1}: Training Loss = {loss:.4f}, Validation Loss = {val_loss[epoch]:.4f}")

# # Plot the training and validation loss
# plt.figure(figsize=(8, 6))
# plt.plot(train_loss, label='Training Loss', marker='o')
# plt.plot(val_loss, label='Validation Loss', marker='o')
# plt.title('Training and Validation Loss Over Epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid()
# plt.show()


In [ ]:
# QUESTIONS
# - scope of project/priorities? one model? multiple? for 3 ppl?
# - AWS GPU credits? need? way to do local or sherlock? 
# - can we do 80/20 split with data we have train/test? do we need dev? 
# 

In [ ]:
# ADVICE
# spend a lot of time on analysis portion - why did this occur if we were moving forward what would we change 
# try predicting TFT time 

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, Input, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, regularizer=None):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim, kernel_regularizer=regularizer)
        self.key_dense = Dense(embed_dim, kernel_regularizer=regularizer)
        self.value_dense = Dense(embed_dim, kernel_regularizer=regularizer)
        self.combine_heads = Dense(embed_dim, kernel_regularizer=regularizer)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True) / tf.sqrt(tf.cast(self.projection_dim, query.dtype))
        weights = tf.nn.softmax(score, axis=-1)
        return tf.matmul(weights, value), weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float32)

        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        attention, _ = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        return self.combine_heads(concat_attention)

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, regularizer=None):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads, regularizer)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu', kernel_regularizer=regularizer),
            Dense(embed_dim, kernel_regularizer=regularizer)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_transformer_model(input_shape, num_heads=4, embed_dim=64, ff_dim=128, num_blocks=2, l2_reg=1e-3):
    inputs = Input(shape=input_shape)
    
    # Embedding layer with regularization
    x = Dense(embed_dim, kernel_regularizer=l2(l2_reg))(inputs)
    
    # Add Transformer blocks with regularization
    for _ in range(num_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=0.3, regularizer=l2(l2_reg))(x, training=True)

    # Global pooling and output layer
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.5)(x) # INCREASED FROM 0.3
    outputs = Dense(1, activation="sigmoid", kernel_regularizer=l2(l2_reg))(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Build the Transformer model
input_shape = (lstm_input.shape[1], lstm_input.shape[2])  # timesteps, features
transformer_model = build_transformer_model(input_shape)

# Summary of the model
transformer_model.summary()


NameError: name 'labels' is not defined

In [15]:
# Train the Transformer model
# from tensorflow.keras.callbacks import EarlyStopping

#early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

## MAYBE TRY BRINGING BACK EARLY STOPPING BUT WITH A DIFFERENT PATIENCE 
## MAYBE TRY CLASS WEIGHTS 

# Compute class weights to balance the dataset
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train), 
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

history = transformer_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,
    batch_size=16,
    #callbacks=[early_stopping],
    class_weight=class_weight_dict,
    verbose=1
)

train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Print final training and validation accuracy
print(f"Final Training Accuracy: {train_accuracy[-1]:.4f}")
print(f"Final Validation Accuracy: {val_accuracy[-1]:.4f}")

# Evaluate the model on test data
test_loss, test_accuracy = transformer_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


NameError: name 'lstm_input' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Plot Training Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()